# Intro
This notebook is a brief introduction into the Open Model Hub. The Open Model Hub makes it extremely easy to apply analytics and data science models on your product data. Below you will find a short demo on how this works. More models are available in the Open Model Hub. The Open Model Hub is not finished yet, more models are added while you are reading this.

The Open Model Hub works if if the full Objectiv stack is used, meaning that data is tracked and stored with the Objectiv Tracker using the Open Taxonomy.

In [ ]:
import sqlalchemy
import os
from bach_open_taxonomy import ObjectivFrame

## Prepare the connection to the data

In [ ]:
from bach_open_taxonomy import ObjectivFrame

## Instantiate the data object
This object points to the data on which the models from the Open Model Hub will be applied. The *time_aggregation* parameter determines the standard timeframe that is used with aggregation functions from the Model Hub. Ie. 'YYYY-MM-DD' means that days are used for the time aggregation. Only data starting at *start_date* is used for all following operations.

In [ ]:
df = ObjectivFrame.from_table(time_aggregation='YYYY-MM-DD', start_date='2021-11-16')

The created object is a Bach DataFrame. Bach is Objectiv’s data modeling library. With Bach, you can use familiar Pandas-like dataframe operations in your notebook. It uses a SQL abstraction layer that enables models to run on the full dataset, and you can output models to SQL with a single command.

The Objectiv Bach api is heavily inspired by the pandas api. We believe this provides a great, generic interface to handle large amounts of data in a python environment while supporting multiple data stores. More information can be found [here](https://objectiv.io/docs/modeling/).

## Using the Open Model Hub
The model hub makes applying models on your data both easy and allows for more advanced models. However, it is possible to use the DataFrame with the regular pandas-like operations that Bach supports, or combine models from the model hub with Bach operations. A more detailed intro of using Objectiv data with Bach, but without the Model Hub is found [here](open-taxonomy-how-to.ipynb).

All models are in the `.model_hub` namespace of the DataFrame. 

### Unique users
Calculating the unique users is one of the models in the model hub. As it is an aggregation model, it is called with `.model_hub.aggregate.unique_users()`. It uses the *time_aggregation* that is set when the DataFrame was instantiated. With `.head()` we immediately query the data to show the first 5 results.

In [ ]:
df.model_hub.aggregate.unique_users().head()

## Combining models
It is also possible to combine models, if the results allow for that. Here unique sessions and unique users are divided to get the average sessions per user per day.

### Average sessions per user per day

In [ ]:
sessions_per_user = df.mh.agg.unique_sessions() / df.mh.agg.unique_users()
sessions_per_user.head()

## Using filters for models
The models above show results for all hits in the data. The data can also be filtered. In this example, we calculate new users per day, instead of all users that we calculated above.

In order to do so, we can use the model hub again to create the filter.

### Unique new users

In [ ]:
is_first_session = df.mh.filter.is_first_session()

In [ ]:
df.mh.agg.unique_users(filter=is_first_session).head()

### Conversion
A filter can also be used for conversions. First has to be defined when a hit is a conversion hit. A conversion is defined by a [location](https://objectiv.io/docs/taxonomy/location-contexts) (on the product) and an event type. The example below defines the clicks to our repository as conversions. A name for the conversion event can optionally be specified.

In [ ]:
df.add_conversion_event(conversion_stack=df.location_stack.json[{'_type': 'LinkContext', 'id': 'cta-repo-button'}:],
                        conversion_event='ClickEvent',
                        name='github_clicks')

Using the conversion, we can calculated the number of converted users at once by creating a filter and using that filter in the unique users model.

In [ ]:
df.mh.agg.unique_users(filter=df.mh.f.conversion('github_clicks')).head()

This is the end of the Open Model Hub introduction. More models are continuously added to the Open Model Hub. Get started with Objectiv and the Open Model Hub [here](https://objectiv.io/).